In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')

# Data Acquisition

In [2]:
df = pd.read_csv('pmsi_analysis_v14.csv')

In [3]:
df.drop('Unnamed: 0',axis=1,inplace=True)

### Provenance coding

In [4]:
def provenance_proc(prov):
    if prov == 5 or prov==5.0 or prov=='5' or prov=='5.0':
        return 'Urgences'
    else:
        return 'Autres'

In [5]:
df['provenance'] = df['provenance'].apply(provenance_proc)

### Dummies Trick

Some data operations in order to get AMO as the base(implicit) case for *raison* generated dummies.

In [6]:
df['raison'] = df['raison'].apply(lambda x: '0 -AMO hors CMU-C' if x=='AMO hors CMU-C' else x)

In [7]:
df['raison'].unique()

array(['AME', '0 -AMO hors CMU-C', 'CMU-C', 'SUV'], dtype=object)

Some data operations in order to get "No severity" as the base (implicit) case for *severity* generated dummies

In [8]:
df['severity'] = df['severity'].astype(str)

In [9]:
df['severity'] = df['severity'].apply(lambda x: '0 - Pas de severité' if x=='Pas de niveau de sévérité' else x)

In [10]:
df['severity'].unique()

array(['2', '3', '1', '0 - Pas de severité', '4'], dtype=object)

# OLS : Regression on Stay Cost

## Cross-Sectionnal Approach

In [15]:
df.columns

Index(['finess', 'mois', 'annee', 'sexe', 'ghm2', 'GHS', 'age', 'duree',
       'supp_rea', 'supp_si', 'supp_stf', 'supp_src', 'supp_nn1', 'supp_nn2',
       'supp_nn3', 'supp_rep', 'ano_date', 'anonyme', 'nbActe', 'nbRum',
       'modeEntree', 'provenance', 'modeSortie', 'motif', 'dp', 'dr', 'cost',
       'raison', 'hp_type', 'severity', 'ghm_racine', 'cmd', 'departement',
       'id_dep', 'region_label', 'population_region', 'Libellé GHM', 'racine',
       'Libellé GHM Racine', 'label_cmd', 'lib_dp', 'region',
       'effectif_region_2020', 'grp_cln'],
      dtype='object')

Let's progress from simple specifications to more comprehensive ones.

$$Y = \beta_{0}+ \beta_{1}\mathbb{1}_{femme} + \beta_{2}age + \beta_{3}age^{2}+ \beta_{4}duree + \epsilon$$ 

In [ ]:
model_eq = "cost ~ C(sexe) + age + np.power(age,2)+ duree"

##### Fitting and Results

In [17]:
model = smf.ols(formula=model_eq,data=df)
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                 3.053e+06
Date:                Sun, 01 May 2022   Prob (F-statistic):               0.00
Time:                        13:27:40   Log-Likelihood:            -3.0018e+07
No. Observations:             3230137   AIC:                         6.004e+07
Df Residuals:                 3230132   BIC:                         6.004e+07
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          256.4292      4.621     55.487      0.000     247.371     265.487
C(sexe)[T.2]       -72.3498      2.941    -24.596      0.000     -78.115     -66.585
age                 21.4233      0.211    101.656      0.000      21.010      21.836
np.power(age, 2)    -0.2532      0.002   -108.743      0.000      -0.258      -0.249
duree              598.1803      0.171   3491.121      0.000     597.844     598.516
==============================================================================
Omnibus:                  6556715.491   Durbin-Watson:                   0.935
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     590161420937.073
Skew:                          15.554   Prob(JB):                         0.00
Kurtosis:                    2096.788   Cond. No.                     1.12e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.12e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Large condition number indicating either a multicolinearity issue ( which seems not to be the case in our specification) or a numerical scaling issue. Let's try to scale $age^2$ by $\frac{1}{1000}$.

$$Y = \beta_{0} + \beta_{1}\mathbb{1}_{femme} + \beta_{2}age + \beta_{3}\frac{age^{2}}{1000}+ \beta_{4}duree + \epsilon$$

In [18]:
model_eq = "cost ~ C(sexe) + age + np.divide(np.power(age,2),1000)+ duree"

In [19]:
model = smf.ols(formula=model_eq,data=df)
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                 3.053e+06
Date:                Sun, 01 May 2022   Prob (F-statistic):               0.00
Time:                        13:52:00   Log-Likelihood:            -3.0018e+07
No. Observations:             3230137   AIC:                         6.004e+07
Df Residuals:                 3230132   BIC:                         6.004e+07
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           256.4292      4.621     55.487      0.000     247.371     265.487
C(sexe)[T.2]                        -72.3498      2.941    -24.596      0.000     -78.115     -66.585
age                                  21.4233      0.211    101.656      0.000      21.010      21.836
np.divide(np.power(age, 2), 1000)  -253.1895      2.328   -108.743      0.000    -257.753    -248.626
duree                               598.1803      0.171   3491.121      0.000     597.844     598.516
==============================================================================
Omnibus:                  6556715.491   Durbin-Watson:                   0.935
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     590161420937.030
Skew:                          15.554   Prob(JB):                         0.00
Kurtosis:                    2096.788   Cond. No.                         175.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

*Large condition number issue is solved by this specification.*

$$Y = \beta_{0} + \beta_{1}\mathbb{1}_{urgences} + \beta_{2}\mathbb{1}_{femme} + \beta_{3}age + \beta_{4}\frac{age^{2}}{1000}+ \beta_{5}duree + \epsilon$$

In [20]:
model_eq = "cost ~ provenance + C(sexe) + age + np.divide(np.power(age,2),1000)+ duree"

In [21]:
model = smf.ols(formula=model_eq,data=df)
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.794
Method:                 Least Squares   F-statistic:                 2.490e+06
Date:                Sun, 01 May 2022   Prob (F-statistic):               0.00
Time:                        13:55:57   Log-Likelihood:            -2.9993e+07
No. Observations:             3230137   AIC:                         5.999e+07
Df Residuals:                 3230131   BIC:                         5.999e+07
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           586.5170      4.817    121.755      0.000     577.075     595.958
provenance[T.Urgences]             -745.1267      3.328   -223.868      0.000    -751.650    -738.603
C(sexe)[T.2]                        -71.7096      2.919    -24.567      0.000     -77.431     -65.989
age                                  15.4272      0.211     73.172      0.000      15.014      15.840
np.divide(np.power(age, 2), 1000)  -204.7059      2.321    -88.213      0.000    -209.254    -200.158
duree                               603.4986      0.172   3515.233      0.000     603.162     603.835
==============================================================================
Omnibus:                  6483732.908   Durbin-Watson:                   0.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     618899514131.696
Skew:                          15.108   Prob(JB):                         0.00
Kurtosis:                    2147.184   Cond. No.                         187.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Adjusted $R^2$ is slightly improved. However the sign for the coefficients of $age$ and $age^2$ seems to contradict the intuition

$$Y =  \beta_{0}  + \beta_{1}\mathbb{1}_{urgences} + \beta_{2}\mathbb{1}_{femme} + \beta_{3}age + \beta_{4}\frac{age^{2}}{1000}+ \beta_{5}duree + \sum_{i=1}^{4} \beta_{5+i}\mathbb{1}_{severity_{i}}+ \epsilon$$

In [41]:
model_eq = "cost ~ provenance + C(sexe) + age + np.divide(np.power(age,2),1000)+ duree + severity"

In [33]:
model = smf.ols(formula=model_eq,data=df)
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.799
Model:                            OLS   Adj. R-squared:                  0.799
Method:                 Least Squares   F-statistic:                 1.430e+06
Date:                Sun, 01 May 2022   Prob (F-statistic):               0.00
Time:                        14:12:27   Log-Likelihood:            -2.9951e+07
No. Observations:             3230137   AIC:                         5.990e+07
Df Residuals:                 3230127   BIC:                         5.990e+07
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           548.7414      4.838    113.413      0.000     539.258     558.225
severity[T.1]                       277.9879      3.957     70.244      0.000     270.231     285.744
severity[T.2]                      -372.6475      5.881    -63.363      0.000    -384.174    -361.121
severity[T.3]                     -1153.3654      7.444   -154.938      0.000   -1167.955   -1138.775
severity[T.4]                      2229.2226     11.808    188.796      0.000    2206.080    2252.365
provenance[T.Urgences]             -716.6953      3.451   -207.649      0.000    -723.460    -709.931
C(sexe)[T.2]                        -51.2453      2.901    -17.667      0.000     -56.930     -45.560
age                                  13.8946      0.208     66.673      0.000      13.486      14.303
np.divide(np.power(age, 2), 1000)  -170.2650      2.307    -73.790      0.000    -174.787    -165.743
duree                               600.1977      0.193   3108.687      0.000     599.819     600.576
==============================================================================
Omnibus:                  6505004.465   Durbin-Watson:                   0.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     662454452935.855
Skew:                          15.211   Prob(JB):                         0.00
Kurtosis:                    2221.362   Cond. No.                         431.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Adjusted $R^2$ is slightly improved. Adding severity dummies do not mitigate the previous sign problem. Moreover, it seems odd that there is a sign change within the estimated gradation of severity ( positive for level 4 and 1 but negative for 2 and 3). It is peculiar that this notion of level gradation is captured by the increased absolute value of estimated coefficient trough the levels.

$$Y =  \beta_{0}  + \beta_{1}\mathbb{1}_{urgences} + \beta_{2}\mathbb{1}_{femme} + \beta_{3}age + \beta_{4}\frac{age^{2}}{1000}+ \beta_{5}duree + \sum_{i=1}^{4} \beta_{5+i}\mathbb{1}_{severity_{i}}+ \sum_{k \in K} \beta_{raison_{k}}\mathbb{1}_{raison_{k}} +\epsilon$$

In [34]:
model_eq = "cost ~ raison + severity + provenance + C(sexe) + age + np.divide(np.power(age,2),1000)+ duree"

In [35]:
model = smf.ols(formula=model_eq,data=df)
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                 1.074e+06
Date:                Sun, 01 May 2022   Prob (F-statistic):               0.00
Time:                        14:45:39   Log-Likelihood:            -2.9949e+07
No. Observations:             3230137   AIC:                         5.990e+07
Df Residuals:                 3230124   BIC:                         5.990e+07
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           616.8085      5.286    116.690      0.000     606.448     627.169
raison[T.AME]                      -138.4900      3.670    -37.731      0.000    -145.684    -131.296
raison[T.CMU-C]                     -47.4135      3.948    -12.011      0.000     -55.151     -39.676
raison[T.SUV]                      -383.0163      7.710    -49.678      0.000    -398.127    -367.905
severity[T.1]                       271.7681      3.975     68.376      0.000     263.978     279.558
severity[T.2]                      -371.9274      5.889    -63.157      0.000    -383.469    -360.385
severity[T.3]                     -1151.2376      7.449   -154.554      0.000   -1165.837   -1136.638
severity[T.4]                      2226.5252     11.809    188.552      0.000    2203.381    2249.670
provenance[T.Urgences]             -705.6754      3.471   -203.328      0.000    -712.478    -698.873
C(sexe)[T.2]                        -53.3800      2.900    -18.405      0.000     -59.065     -47.695
age                                  15.7546      0.211     74.546      0.000      15.340      16.169
np.divide(np.power(age, 2), 1000)  -201.4180      2.394    -84.127      0.000    -206.111    -196.725
duree                               600.8888      0.193   3107.090      0.000     600.510     601.268
==============================================================================
Omnibus:                  6494779.522   Durbin-Watson:                   0.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     664551777119.601
Skew:                          15.150   Prob(JB):                         0.00
Kurtosis:                    2224.873   Cond. No.                         432.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Adjusted $R^2$ is slightly improved. Previous problems are maintained. Negative signs for the coefficients corresponding to the system dummies. Increased absolute value of estimated coefficient trough the levels of precariousness ( from CMU-C, to AME and SUV)

In [36]:
df.columns

Index(['finess', 'mois', 'annee', 'sexe', 'ghm2', 'GHS', 'age', 'duree',
       'supp_rea', 'supp_si', 'supp_stf', 'supp_src', 'supp_nn1', 'supp_nn2',
       'supp_nn3', 'supp_rep', 'ano_date', 'anonyme', 'nbActe', 'nbRum',
       'modeEntree', 'provenance', 'modeSortie', 'motif', 'dp', 'dr', 'cost',
       'raison', 'hp_type', 'severity', 'ghm_racine', 'cmd', 'departement',
       'id_dep', 'region_label', 'population_region', 'Libellé GHM', 'racine',
       'Libellé GHM Racine', 'label_cmd', 'lib_dp', 'region',
       'effectif_region_2020', 'grp_cln'],
      dtype='object')

$$Y =  \beta_{0}  + \beta_{1}\mathbb{1}_{urgences} + \beta_{2}\mathbb{1}_{femme} + \beta_{3}age + \beta_{4}\frac{age^{2}}{1000}+ \beta_{5}duree + \sum_{i=1}^{4} \beta_{5+i}\mathbb{1}_{severity_{i}}+ \sum_{k \in K} \beta_{raison_{k}}\mathbb{1}_{raison_{k}}+ \sum_{s \in S} \beta_{supp_{s}}supp_{s}+ \epsilon$$

In [39]:
model_eq = '''cost ~ raison + severity + provenance + C(sexe)+ age 
+ np.divide(np.power(age,2),1000)+ duree + supp_rea + supp_si + supp_stf+
supp_src + supp_nn1 + supp_nn2 + supp_nn3 + supp_rep'''

In [40]:
model = smf.ols(formula=model_eq,data=df)
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                 1.275e+06
Date:                Sun, 01 May 2022   Prob (F-statistic):               0.00
Time:                        14:57:58   Log-Likelihood:            -2.9016e+07
No. Observations:             3230137   AIC:                         5.803e+07
Df Residuals:                 3230116   BIC:                         5.803e+07
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           570.7285      4.001    142.629      0.000     562.886     578.571
raison[T.AME]                      -105.2202      2.749    -38.270      0.000    -110.609     -99.831
raison[T.CMU-C]                     -53.4731      2.957    -18.086      0.000     -59.268     -47.678
raison[T.SUV]                      -257.5807      5.776    -44.596      0.000    -268.901    -246.260
severity[T.1]                       381.9800      2.981    128.149      0.000     376.138     387.822
severity[T.2]                       -24.0016      4.425     -5.424      0.000     -32.675     -15.328
severity[T.3]                      -720.5221      5.608   -128.474      0.000    -731.514    -709.530
severity[T.4]                       181.2924      9.067     19.994      0.000     163.521     199.064
provenance[T.Urgences]             -581.8572      2.605   -223.404      0.000    -586.962    -576.752
C(sexe)[T.2]                         29.2092      2.173     13.440      0.000      24.950      33.469
age                                  15.0895      0.160     94.168      0.000      14.775      15.404
np.divide(np.power(age, 2), 1000)  -169.2556      1.808    -93.592      0.000    -172.800    -165.711
duree                               516.3784      0.161   3197.942      0.000     516.062     516.695
supp_rea                            749.5444      0.627   1195.234      0.000     748.315     750.773
supp_si                             -74.9402      2.849    -26.301      0.000     -80.525     -69.356
supp_stf                            549.7075      0.718    765.716      0.000     548.300     551.115
supp_src                           -145.8212      0.850   -171.480      0.000    -147.488    -144.154
supp_nn1                            -53.3344      1.104    -48.301      0.000     -55.499     -51.170
supp_nn2                            159.7793      1.415    112.955      0.000     157.007     162.552
supp_nn3                            680.2271      1.338    508.273      0.000     677.604     682.850
supp_rep                            991.1533      1.597    620.679      0.000     988.023     994.283
==============================================================================
Omnibus:                  5084816.047   Durbin-Watson:                   0.757
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     108504812462.226
Skew:                           8.823   Prob(JB):                         0.00
Kurtosis:                     900.709   Cond. No.                         442.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Adjusted $R^2$ improved (makes senses as supplement are a part of the actual cost computation algorithm). Previous problems are maintained. Moreover, supp_si and supp_src have a negative which doesn't make sense as far as cost calculation is concerned. There is probably,as for the other cases, one or multiple instances of the following issues : 

* Omiited Variables
* Selection Bias (low probability given the dataset generation)
* Outliers 


$$Y =  \beta_{0}  + \beta_{1}\mathbb{1}_{urgences} + \beta_{2}\mathbb{1}_{femme} + \beta_{3}age + \beta_{4}\frac{age^{2}}{1000}+ \beta_{5}duree + \sum_{i=1}^{4} \beta_{5+i}\mathbb{1}_{severity_{i}}+ \sum_{k \in K} \beta_{raison_{k}}\mathbb{1}_{raison_{k}}+ \sum_{s \in S} \beta_{supp_{s}}supp_{s}+\sum_{j=1,j\neq 24}^{28} \beta_{cmd_{j}}\mathbb{1}_{cmd_{j}}+ \epsilon$$

In [60]:
df['ones'] = 1

In [71]:
df.columnsumns

Index(['finess', 'mois', 'annee', 'sexe', 'ghm2', 'GHS', 'age', 'duree',
       'supp_rea', 'supp_si', 'supp_stf', 'supp_src', 'supp_nn1', 'supp_nn2',
       'supp_nn3', 'supp_rep', 'ano_date', 'anonyme', 'nbActe', 'nbRum',
       'modeEntree', 'provenance', 'modeSortie', 'motif', 'dp', 'dr', 'cost',
       'raison', 'hp_type', 'severity', 'ghm_racine', 'cmd', 'departement',
       'id_dep', 'region_label', 'population_region', 'Libellé GHM', 'racine',
       'Libellé GHM Racine', 'label_cmd', 'lib_dp', 'region',
       'effectif_region_2020', 'grp_cln', 'ones'],
      dtype='object')

In [69]:
model_eq = '''cost ~ raison + severity + provenance + C(sexe)+ age 
+ np.divide(np.power(age,2),1000)+ duree + supp_rea + supp_si + supp_stf+
supp_src + supp_nn1 + supp_nn2 + supp_nn3 + supp_rep + C(cmd)'''

In [70]:
model = smf.ols(formula=model_eq,data=df)
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.895
Method:                 Least Squares   F-statistic:                 5.981e+05
Date:                Sun, 01 May 2022   Prob (F-statistic):               0.00
Time:                        16:04:46   Log-Likelihood:            -2.8906e+07
No. Observations:             3230137   AIC:                         5.781e+07
Df Residuals:                 3230090   BIC:                         5.781e+07
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           679.3625      6.505    104.443      0.000     666.614     692.111
raison[T.AME]                      -140.5185      2.693    -52.185      0.000    -145.796    -135.241
raison[T.CMU-C]                     -91.1540      2.886    -31.586      0.000     -96.810     -85.498
raison[T.SUV]                      -277.3274      5.610    -49.433      0.000    -288.323    -266.332
severity[T.1]                       186.6689      3.337     55.943      0.000     180.129     193.209
severity[T.2]                      -179.5447      4.646    -38.647      0.000    -188.650    -170.439
severity[T.3]                      -826.0249      5.752   -143.607      0.000    -837.299    -814.751
severity[T.4]                        27.1063      8.980      3.019      0.003       9.507      44.706
provenance[T.Urgences]             -505.9796      2.726   -185.582      0.000    -511.323    -500.636
C(sexe)[T.2]                         56.4796      2.253     25.073      0.000      52.065      60.895
C(cmd)[T.2]                         846.8067      9.080     93.259      0.000     829.010     864.604
C(cmd)[T.3]                         521.1513      7.913     65.859      0.000     505.642     536.661
C(cmd)[T.4]                        -237.1727      6.632    -35.763      0.000    -250.171    -224.175
C(cmd)[T.5]                         900.3581      6.531    137.853      0.000     887.557     913.159
C(cmd)[T.6]                         359.1156      6.176     58.145      0.000     347.010     371.221
C(cmd)[T.7]                         289.2738      8.493     34.060      0.000     272.628     305.920
C(cmd)[T.8]                         932.1211      6.507    143.253      0.000     919.368     944.874
C(cmd)[T.9]                         324.1925      7.801     41.559      0.000     308.903     339.482
C(cmd)[T.10]                        -53.5220      7.962     -6.722      0.000     -69.127     -37.917
C(cmd)[T.11]                        326.8475      7.873     41.517      0.000     311.418     342.277
C(cmd)[T.12]                        494.7995     12.366     40.014      0.000     470.563     519.036
C(cmd)[T.13]                        944.0070      9.196    102.654      0.000     925.983     962.031
C(cmd)[T.14]                        -57.2735      6.195     -9.245      0.000     -69.416     -45.131
C(cmd)[T.15]                      -1345.8879      9.582   -140.463      0.000   -1364.668   -1327.108
C(cmd)[T.16]                        385.2730      9.940     38.759      0.000     365.791     404.755
C(cmd)[T.17]                        784.7640     10.227     76.737      0.000     764.720     804.808
C(cmd)[T.18]                        175.3574     11.510     15.235      0.000     152.798     197.917
C(cmd)[T.19]           

Some correction of the coefficient of $age$ and $age^2$ in the right direction.

$$Y =  \beta_{0}  + \beta_{1}\mathbb{1}_{urgences} + \beta_{2}\mathbb{1}_{femme} + \beta_{3}age + \beta_{4}\frac{age^{2}}{1000}+ \beta_{5}duree + \sum_{i=1}^{4} \beta_{5+i}\mathbb{1}_{severity_{i}}+ \sum_{k \in K} \beta_{raison_{k}}\mathbb{1}_{raison_{k}}+ \sum_{s \in S} \beta_{supp_{s}}supp_{s}+\sum_{g\in G}\beta_{ghm_{g}}\mathbb{1}_{ghm_{g}}+ \epsilon$$

In [74]:
df.columns

Index(['finess', 'mois', 'annee', 'sexe', 'ghm2', 'GHS', 'age', 'duree',
       'supp_rea', 'supp_si', 'supp_stf', 'supp_src', 'supp_nn1', 'supp_nn2',
       'supp_nn3', 'supp_rep', 'ano_date', 'anonyme', 'nbActe', 'nbRum',
       'modeEntree', 'provenance', 'modeSortie', 'motif', 'dp', 'dr', 'cost',
       'raison', 'hp_type', 'severity', 'ghm_racine', 'cmd', 'departement',
       'id_dep', 'region_label', 'population_region', 'Libellé GHM', 'racine',
       'Libellé GHM Racine', 'label_cmd', 'lib_dp', 'region',
       'effectif_region_2020', 'grp_cln', 'ones'],
      dtype='object')

In [76]:
model_eq = '''cost ~ raison + severity + provenance + C(sexe)+ age 
+ np.divide(np.power(age,2),1000)+ duree + supp_rea + supp_si + supp_stf+
supp_src + supp_nn1 + supp_nn2 + supp_nn3 + supp_rep + ghm2'''

In [80]:
model = smf.ols(formula=model_eq,data=df.sample(int(8e5)))
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.959
Method:                 Least Squares   F-statistic:                     7692.
Date:                Sun, 01 May 2022   Prob (F-statistic):               0.00
Time:                        16:30:04   Log-Likelihood:            -6.7738e+06
No. Observations:              799979   AIC:                         1.355e+07
Df Residuals:                  797562   BIC:                         1.358e+07
Df Model:                        2416                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1038.8521      9.358    111.008      0.000    1020.510    1057.194
raison[T.AME]                        11.4531      3.432      3.337      0.001       4.726      18.180
raison[T.CMU-C]                     -18.1925      3.617     -5.029      0.000     -25.282     -11.103
raison[T.SUV]                        -5.1323      7.097     -0.723      0.470     -19.043       8.778
severity[T.1]                       627.9200    191.955      3.271      0.001     251.694    1004.146
severity[T.2]                       759.3105     15.235     49.839      0.000     729.450     789.171
severity[T.3]                       757.0082     20.068     37.723      0.000     717.676     796.340
severity[T.4]                      2474.5049     25.814     95.860      0.000    2423.911    2525.099
provenance[T.Urgences]               22.5054      4.093      5.499      0.000      14.483      30.527
C(sexe)[T.2]                         11.3418      2.878      3.941      0.000       5.701      16.983
ghm2[T.01C032]                     -254.9604    154.335     -1.652      0.099    -557.452      47.531
ghm2[T.01C033]                      -87.9103    167.102     -0.526      0.599    -415.425     239.604
ghm2[T.01C034]                    -1425.9616    129.541    -11.008      0.000   -1679.858   -1172.065
ghm2[T.01C041]                     1617.6331    207.415      7.799      0.000    1211.107    2024.159
ghm2[T.01C042]                     1720.7392     71.181     24.174      0.000    1581.228    1860.251
ghm2[T.01C043]                     1514.0398     84.087     18.006      0.000    1349.231    1678.848
ghm2[T.01C044]                     1917.8059     83.962     22.841      0.000    1753.244    2082.368
ghm2[T.01C051]                     1870.9538    213.180      8.776      0.000    1453.129    2288.779
ghm2[T.01C052]                     1789.6498    111.522     16.047      0.000    1571.070    2008.230
ghm2[T.01C053]                     2158.6801    129.957     16.611      0.000    1903.969    2413.391
ghm2[T.01C054]                    -1012.4771    236.075     -4.289      0.000   -1475.177    -549.777
ghm2[T.01C061]                     1101.7417    214.547      5.135      0.000     681.236    1522.247
ghm2[T.01C062]                     1153.3461    127.006      9.081      0.000     904.418    1402.274
ghm2[T.01C063]                     1049.9983    207.458      5.061      0.000     643.387    1456.609
ghm2[T.01C064]                    -1964.9764   1150.384     -1.708      0.088   -4219.691     289.738
ghm2[T.01C081]                      516.9368    203.393      2.542      0.011     118.293     915.581
ghm2[T.01C082]                      495.7099    192.217      2.579      0.010     118.971     872.449
ghm2[T.01C083]         

Controlling for $ghm$ makes the coefficients of interest go clearly in the right direction ( although not completly). Let's try the same approach but with a less granular variable : $ghm racine$

df

In [84]:
df['racine'].nunique()

649

$$Y =  \beta_{0}  + \beta_{1}\mathbb{1}_{urgences} + \beta_{2}\mathbb{1}_{femme} + \beta_{3}age + \beta_{4}\frac{age^{2}}{1000}+ \beta_{5}duree + \sum_{i=1}^{4} \beta_{5+i}\mathbb{1}_{severity_{i}}+ \sum_{k \in K} \beta_{raison_{k}}\mathbb{1}_{raison_{k}}+ \sum_{s \in S} \beta_{supp_{s}}supp_{s}+\sum_{g\in R}\beta_{rac_{g}}\mathbb{1}_{rac_{g}}+ \epsilon$$

In [74]:
df.columns

Index(['finess', 'mois', 'annee', 'sexe', 'ghm2', 'GHS', 'age', 'duree',
       'supp_rea', 'supp_si', 'supp_stf', 'supp_src', 'supp_nn1', 'supp_nn2',
       'supp_nn3', 'supp_rep', 'ano_date', 'anonyme', 'nbActe', 'nbRum',
       'modeEntree', 'provenance', 'modeSortie', 'motif', 'dp', 'dr', 'cost',
       'raison', 'hp_type', 'severity', 'ghm_racine', 'cmd', 'departement',
       'id_dep', 'region_label', 'population_region', 'Libellé GHM', 'racine',
       'Libellé GHM Racine', 'label_cmd', 'lib_dp', 'region',
       'effectif_region_2020', 'grp_cln', 'ones'],
      dtype='object')

In [11]:
model_eq = '''cost ~ raison + severity + provenance + C(sexe)+ age 
+ np.divide(np.power(age,2),1000)+ duree + supp_rea + supp_si + supp_stf+
supp_src + supp_nn1 + supp_nn2 + supp_nn3 + supp_rep + racine'''

In [12]:
model = smf.ols(formula=model_eq,data=df.sample(int(1e6)))
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.950
Method:                 Least Squares   F-statistic:                 2.867e+04
Date:                Mon, 02 May 2022   Prob (F-statistic):               0.00
Time:                        00:33:21   Log-Likelihood:            -8.5769e+06
No. Observations:              999974   AIC:                         1.716e+07
Df Residuals:                  999306   BIC:                         1.716e+07
Df Model:                         667                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1637.8644     74.413     22.011      0.000    1492.018    1783.711
raison[T.AME]                         7.6984      3.410      2.258      0.024       1.016      14.381
raison[T.CMU-C]                     -23.6973      3.601     -6.581      0.000     -30.755     -16.640
raison[T.SUV]                       -16.6697      7.038     -2.368      0.018     -30.465      -2.875
severity[T.1]                      -368.8652      5.278    -69.882      0.000    -379.211    -358.520
severity[T.2]                      -503.7761      6.634    -75.938      0.000    -516.779    -490.774
severity[T.3]                      -870.9012      7.982   -109.114      0.000    -886.545    -855.258
severity[T.4]                        -5.2378     11.980     -0.437      0.662     -28.717      18.242
provenance[T.Urgences]                0.8219      3.993      0.206      0.837      -7.005       8.649
C(sexe)[T.2]                         12.5005      2.863      4.367      0.000       6.890      18.111
racine[T.01C04]                    2358.8078     82.884     28.459      0.000    2196.359    2521.257
racine[T.01C05]                    2265.1052     94.317     24.016      0.000    2080.248    2449.963
racine[T.01C06]                    1590.1624    102.274     15.548      0.000    1389.709    1790.615
racine[T.01C08]                     764.4593     89.291      8.561      0.000     589.451     939.467
racine[T.01C09]                    1.821e+04    192.945     94.383      0.000    1.78e+04    1.86e+04
racine[T.01C10]                    7933.3925    132.410     59.915      0.000    7673.873    8192.913
racine[T.01C11]                    4223.8621    157.909     26.749      0.000    3914.365    4533.359
racine[T.01C12]                    3239.7823    118.917     27.244      0.000    3006.709    3472.855
racine[T.01C14]                    -107.5122    112.404     -0.956      0.339    -327.821     112.797
racine[T.01C15]                    -580.3842     81.946     -7.083      0.000    -740.995    -419.774
racine[T.01K02]                    8041.7932    116.454     69.055      0.000    7813.547    8270.040
racine[T.01K03]                    2062.5451    116.127     17.761      0.000    1834.940    2290.150
racine[T.01K04]                   -1118.6901     79.737    -14.030      0.000   -1274.973    -962.408
racine[T.01K05]                   -1335.7296    157.873     -8.461      0.000   -1645.155   -1026.305
racine[T.01K06]                    -340.6507     97.103     -3.508      0.000    -530.970    -150.331
racine[T.01K07]                    6005.4122    159.330     37.692      0.000    5693.130    6317.694
racine[T.01M04]                    -871.5818     96.354     -9.046      0.000   -1060.433    -682.731
racine[T.01M05]        

$$Y =  \beta_{0}  + \beta_{1}\mathbb{1}_{urgences} + \beta_{2}\mathbb{1}_{femme} + \beta_{3}age + \beta_{4}\frac{age^{2}}{1000}+ \beta_{5}duree + \sum_{k \in K} \beta_{raison_{k}}\mathbb{1}_{raison_{k}}+ \sum_{s \in S} \beta_{supp_{s}}supp_{s}+\sum_{g\in R}\beta_{rac_{g}}\mathbb{1}_{rac_{g}}+ \epsilon$$

In [13]:
df.columns

Index(['finess', 'mois', 'annee', 'sexe', 'ghm2', 'GHS', 'age', 'duree',
       'supp_rea', 'supp_si', 'supp_stf', 'supp_src', 'supp_nn1', 'supp_nn2',
       'supp_nn3', 'supp_rep', 'ano_date', 'anonyme', 'nbActe', 'nbRum',
       'modeEntree', 'provenance', 'modeSortie', 'motif', 'dp', 'dr', 'cost',
       'raison', 'hp_type', 'severity', 'ghm_racine', 'cmd', 'departement',
       'id_dep', 'region_label', 'population_region', 'Libellé GHM', 'racine',
       'Libellé GHM Racine', 'label_cmd', 'lib_dp', 'region',
       'effectif_region_2020', 'grp_cln'],
      dtype='object')

In [14]:
model_eq = '''cost ~ raison + provenance + C(sexe)+ age 
+ np.divide(np.power(age,2),1000)+ duree + supp_rea + supp_si + supp_stf+
supp_src + supp_nn1 + supp_nn2 + supp_nn3 + supp_rep + racine'''

In [15]:
model = smf.ols(formula=model_eq,data=df.sample(int(1e6)))
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.951
Method:                 Least Squares   F-statistic:                 2.935e+04
Date:                Mon, 02 May 2022   Prob (F-statistic):               0.00
Time:                        00:39:53   Log-Likelihood:            -8.5758e+06
No. Observations:              999973   AIC:                         1.715e+07
Df Residuals:                  999309   BIC:                         1.716e+07
Df Model:                         663                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                          1516.7631     74.012     20.493      0.000    1371.702    1661.825
raison[T.AME]                        -0.8422      3.405     -0.247      0.805      -7.516       5.832
raison[T.CMU-C]                     -30.3895      3.595     -8.454      0.000     -37.435     -23.344
raison[T.SUV]                       -25.8723      7.032     -3.679      0.000     -39.654     -12.090
provenance[T.Urgences]              -16.9775      3.985     -4.261      0.000     -24.787      -9.168
C(sexe)[T.2]                         11.1573      2.860      3.902      0.000       5.552      16.762
racine[T.01C04]                    2185.5349     82.875     26.371      0.000    2023.102    2347.968
racine[T.01C05]                    2187.6297     95.854     22.823      0.000    1999.760    2375.500
racine[T.01C06]                    1356.7365    101.986     13.303      0.000    1156.848    1556.625
racine[T.01C08]                     640.9447     88.819      7.216      0.000     466.863     815.026
racine[T.01C09]                    1.845e+04    191.081     96.576      0.000    1.81e+04    1.88e+04
racine[T.01C10]                    6974.4953    128.411     54.314      0.000    6722.815    7226.176
racine[T.01C11]                    3770.2403    149.286     25.255      0.000    3477.645    4062.835
racine[T.01C12]                    2568.8864    117.175     21.924      0.000    2339.228    2798.545
racine[T.01C14]                     -31.1165    109.405     -0.284      0.776    -245.547     183.314
racine[T.01C15]                    -476.4900     81.734     -5.830      0.000    -636.685    -316.295
racine[T.01K02]                    7318.2153    120.951     60.505      0.000    7081.154    7555.276
racine[T.01K03]                    2059.4803    116.888     17.619      0.000    1830.385    2288.576
racine[T.01K04]                    -982.7652     79.178    -12.412      0.000   -1137.951    -827.579
racine[T.01K05]                   -1203.3466    158.265     -7.603      0.000   -1513.541    -893.153
racine[T.01K06]                    -214.6480     97.484     -2.202      0.028    -405.713     -23.583
racine[T.01K07]                    6340.1715    161.348     39.295      0.000    6023.936    6656.407
racine[T.01M04]                   -1065.3242     96.168    -11.078      0.000   -1253.809    -876.839
racine[T.01M05]                      68.7981     88.114      0.781      0.435    -103.902     241.498
racine[T.01M07]                   -1829.8659    105.922    -17.276      0.000   -2037.470   -1622.262
racine[T.01M08]                   -1369.3989     81.540    -16.794      0.000   -1529.214   -1209.583
racine[T.01M09]                   -1182.2548    112.462    -10.512      0.000   -1402.677    -961.832
racine[T.01M10]        

$$Y =  \beta_{0}  + \beta_{1}\mathbb{1}_{urgences} + \beta_{2}\mathbb{1}_{femme} + \beta_{3}age + \beta_{4}\frac{age^{2}}{1000}+ \beta_{5}duree + \sum_{k \in K} \beta_{raison_{k}}\mathbb{1}_{raison_{k}}+ \sum_{s \in S} \beta_{supp_{s}}supp_{s}+\sum_{g\in T}\beta_{rac_{g}}\mathbb{1}_{rac_{g}}+ \sum_{i\in R}\beta_{region_{i}}\mathbb{1}_{region_{i}}+ \epsilon$$

In [13]:
df.columns

Index(['finess', 'mois', 'annee', 'sexe', 'ghm2', 'GHS', 'age', 'duree',
       'supp_rea', 'supp_si', 'supp_stf', 'supp_src', 'supp_nn1', 'supp_nn2',
       'supp_nn3', 'supp_rep', 'ano_date', 'anonyme', 'nbActe', 'nbRum',
       'modeEntree', 'provenance', 'modeSortie', 'motif', 'dp', 'dr', 'cost',
       'raison', 'hp_type', 'severity', 'ghm_racine', 'cmd', 'departement',
       'id_dep', 'region_label', 'population_region', 'Libellé GHM', 'racine',
       'Libellé GHM Racine', 'label_cmd', 'lib_dp', 'region',
       'effectif_region_2020', 'grp_cln'],
      dtype='object')

In [16]:
model_eq = '''cost ~ raison + provenance + C(sexe)+ age 
+ np.divide(np.power(age,2),1000)+ duree + supp_rea + supp_si + supp_stf+
supp_src + supp_nn1 + supp_nn2 + supp_nn3 + supp_rep + racine + region'''

In [17]:
model = smf.ols(formula=model_eq,data=df.sample(int(2e6)))
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.950
Method:                 Least Squares   F-statistic:                 5.553e+04
Date:                Mon, 02 May 2022   Prob (F-statistic):               0.00
Time:                        00:51:24   Log-Likelihood:            -1.7114e+07
No. Observations:             1999949   AIC:                         3.423e+07
Df Residuals:                 1999268   BIC:                         3.424e+07
Df Model:                         680                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                             1545.6595     51.155     30.215      0.000    1445.398    1645.921
raison[T.AME]                            1.2380      2.483      0.499      0.618      -3.629       6.105
raison[T.CMU-C]                        -34.6806      2.513    -13.800      0.000     -39.606     -29.755
raison[T.SUV]                          -27.9539      5.244     -5.330      0.000     -38.233     -17.675
provenance[T.Urgences]                 -17.4660      2.784     -6.274      0.000     -22.922     -12.009
C(sexe)[T.2]                            10.5866      1.985      5.333      0.000       6.696      14.477
racine[T.01C04]                       2236.4464     57.148     39.134      0.000    2124.439    2348.454
racine[T.01C05]                       1961.1409     65.998     29.715      0.000    1831.788    2090.494
racine[T.01C06]                       1356.9535     70.292     19.305      0.000    1219.184    1494.723
racine[T.01C08]                        602.9027     61.377      9.823      0.000     482.606     723.199
racine[T.01C09]                       1.694e+04    130.419    129.880      0.000    1.67e+04    1.72e+04
racine[T.01C10]                       7073.4805     91.190     77.568      0.000    6894.751    7252.210
racine[T.01C11]                       4178.7827    109.715     38.088      0.000    3963.746    4393.819
racine[T.01C12]                       3074.3628     81.931     37.524      0.000    2913.782    3234.944
racine[T.01C14]                        -60.7423     77.842     -0.780      0.435    -213.311      91.826
racine[T.01C15]                       -491.3313     56.304     -8.726      0.000    -601.686    -380.977
racine[T.01K02]                       7232.8611     82.280     87.906      0.000    7071.596    7394.127
racine[T.01K03]                       1966.1263     80.861     24.315      0.000    1807.641    2124.612
racine[T.01K04]                      -1002.4247     54.822    -18.285      0.000   -1109.874    -894.975
racine[T.01K05]                      -1233.9978    112.393    -10.979      0.000   -1454.285   -1013.711
racine[T.01K06]                       -245.0601     66.961     -3.660      0.000    -376.301    -113.819
racine[T.01K07]                       6289.2309    106.305     59.162      0.000    6080.877    6497.585
racine[T.01M04]                      -1089.5976     66.583    -16.365      0.000   -1220.097    -959.098
racine[T.01M05]                        179.5847     60.551      2.966      0.003      60.907     298.262
racine[T.01M07]                      -1788.3181     72.199    -24.769      0.000   -1929.826   -1646.810
racine[T.01M08]                      -1410.4210     56.465    -24.978      0.000   -1521.091   -1299.751
racine[T.01M09]                      -

## Repeated Cross Section

In [18]:
df['annee'].unique()

array([2012, 2015, 2017, 2019, 2020, 2021, 2013, 2014, 2016, 2018, 2011],
      dtype=int64)

$$Y =  \beta_{0}  + \beta_{1}\mathbb{1}_{urgences} + \beta_{2}\mathbb{1}_{femme} + \beta_{3}age + \beta_{4}\frac{age^{2}}{1000}+ \beta_{5}duree + \sum_{i=1}^{4} \beta_{5+i}\mathbb{1}_{severity_{i}}+ \sum_{k \in K} \beta_{raison_{k}}\mathbb{1}_{raison_{k}} + \sum_{y=2011}^{2021}\beta_{year_{y}}\mathbb{1}_{year_{y}}+\epsilon$$

In [26]:
model_eq = "cost ~ C(annee) + raison + severity + provenance + C(sexe) + age + np.divide(np.power(age,2),1000)+ duree"

In [27]:
model = smf.ols(formula=model_eq,data=df)
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                 5.860e+05
Date:                Mon, 02 May 2022   Prob (F-statistic):               0.00
Time:                        01:06:23   Log-Likelihood:            -2.9949e+07
No. Observations:             3230137   AIC:                         5.990e+07
Df Residuals:                 3230114   BIC:                         5.990e+07
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                           484.1315     12.212     39.645      0.000     460.197     508.066
C(annee)[T.2012]                     76.9358     12.082      6.368      0.000      53.256     100.616
C(annee)[T.2013]                     49.6135     11.932      4.158      0.000      26.227      73.000
C(annee)[T.2014]                     95.2026     12.066      7.890      0.000      71.554     118.851
C(annee)[T.2015]                    126.1699     12.019     10.498      0.000     102.614     149.726
C(annee)[T.2016]                    135.0815     11.945     11.308      0.000     111.669     158.494
C(annee)[T.2017]                    138.4346     11.912     11.621      0.000     115.087     161.782
C(annee)[T.2018]                    151.4512     11.877     12.752      0.000     128.173     174.730
C(annee)[T.2019]                    155.1919     11.820     13.130      0.000     132.026     178.358
C(annee)[T.2020]                    181.7337     11.861     15.321      0.000     158.486     204.982
C(annee)[T.2021]                    234.2705     11.767     19.909      0.000     211.208     257.333
raison[T.AME]                      -136.9871      3.712    -36.904      0.000    -144.263    -129.712
raison[T.CMU-C]                     -50.1422      3.987    -12.576      0.000     -57.957     -42.327
raison[T.SUV]                      -379.1402      7.735    -49.018      0.000    -394.300    -363.981
severity[T.1]                       284.2894      3.991     71.225      0.000     276.466     292.112
severity[T.2]                      -367.5140      5.890    -62.391      0.000    -379.059    -355.969
severity[T.3]                     -1149.0559      7.450   -154.245      0.000   -1163.657   -1134.455
severity[T.4]                      2221.3188     11.809    188.107      0.000    2198.174    2244.464
provenance[T.Urgences]             -711.2826      3.487   -203.981      0.000    -718.117    -704.448
C(sexe)[T.2]                        -51.8647      2.900    -17.883      0.000     -57.549     -46.180
age                                  15.5466      0.211     73.545      0.000      15.132      15.961
np.divide(np.power(age, 2), 1000)  -200.1279      2.394    -83.579      0.000    -204.821    -195.435
duree                               601.0634      0.193   3107.499      0.000     600.684     601.442
==============================================================================
Omnibus:                  6493019.571   Durbin-Watson:                   0.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     665921815165.307
Skew:                          15.139   Prob(JB):                         0.00
Kurtosis:                    2227.163   Cond. No.                     1.32e+03
====================================

$$Y =  \beta_{0}  + \beta_{1}\mathbb{1}_{urgences} + \beta_{2}\mathbb{1}_{femme} + \beta_{3}age + \beta_{4}\frac{age^{2}}{1000}+ \beta_{5}duree + \sum_{k \in K} \beta_{raison_{k}}\mathbb{1}_{raison_{k}}+ \sum_{s \in S} \beta_{supp_{s}}supp_{s}+\sum_{g\in T}\beta_{rac_{g}}\mathbb{1}_{rac_{g}}+ \sum_{i\in R}\beta_{region_{i}}\mathbb{1}_{region_{i}}+\sum_{y=2011}^{2021}\beta_{year_{y}}\mathbb{1}_{year_{y}}+ \epsilon$$

In [28]:
model_eq = '''cost ~ C(annee) + raison + provenance + C(sexe)+ age 
+ np.divide(np.power(age,2),1000)+ duree + supp_rea + supp_si + supp_stf+
supp_src + supp_nn1 + supp_nn2 + supp_nn3 + supp_rep + racine + region'''

In [29]:
model = smf.ols(formula=model_eq,data=df.sample(int(2e6)))
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cost   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.950
Method:                 Least Squares   F-statistic:                 5.561e+04
Date:                Mon, 02 May 2022   Prob (F-statistic):               0.00
Time:                        01:20:44   Log-Likelihood:            -1.7138e+07
No. Observations:             1999959   AIC:                         3.428e+07
Df Residuals:                 1999268   BIC:                         3.429e+07
Df Model:                         690                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                             1524.8942     53.685     28.404      0.000    1419.673    1630.115
C(annee)[T.2012]                       -32.9427      7.639     -4.312      0.000     -47.915     -17.970
C(annee)[T.2013]                       -37.2057      7.557     -4.924      0.000     -52.016     -22.395
C(annee)[T.2014]                       -25.1273      7.636     -3.291      0.001     -40.094     -10.161
C(annee)[T.2015]                       -29.7670      7.609     -3.912      0.000     -44.680     -14.854
C(annee)[T.2016]                       -23.9381      7.564     -3.165      0.002     -38.763      -9.114
C(annee)[T.2017]                       -34.7035      7.545     -4.599      0.000     -49.492     -19.915
C(annee)[T.2018]                       -33.3360      7.523     -4.431      0.000     -48.082     -18.590
C(annee)[T.2019]                       -37.5978      7.489     -5.020      0.000     -52.276     -22.920
C(annee)[T.2020]                       -40.1464      7.521     -5.338      0.000     -54.887     -25.406
C(annee)[T.2021]                       -27.2263      7.461     -3.649      0.000     -41.850     -12.602
raison[T.AME]                            3.4823      2.536      1.373      0.170      -1.488       8.453
raison[T.CMU-C]                        -30.5073      2.569    -11.873      0.000     -35.543     -25.471
raison[T.SUV]                          -35.2491      5.320     -6.626      0.000     -45.676     -24.822
provenance[T.Urgences]                 -11.5507      2.834     -4.075      0.000     -17.106      -5.995
C(sexe)[T.2]                             8.1062      2.009      4.036      0.000       4.170      12.043
racine[T.01C04]                       2260.0088     59.280     38.124      0.000    2143.822    2376.196
racine[T.01C05]                       2105.2491     67.937     30.988      0.000    1972.095    2238.403
racine[T.01C06]                       1410.3375     72.873     19.353      0.000    1267.508    1553.167
racine[T.01C08]                        627.0267     63.920      9.809      0.000     501.745     752.308
racine[T.01C09]                       1.891e+04    137.455    137.577      0.000    1.86e+04    1.92e+04
racine[T.01C10]                       7704.6545     93.244     82.629      0.000    7521.900    7887.409
racine[T.01C11]                       3791.7309    111.273     34.076      0.000    3573.640    4009.822
racine[T.01C12]                       3116.6223     82.288     37.874      0.000    2955.340    3277.905
racine[T.01C14]                         -1.8007     79.287     -0.023      0.982    -157.201     153.599
racine[T.01C15]                       -449.6923     58.294     -7.714      0.000    -563.946    -335.438
racine[T.01K02]                       